#  Communication Vis - Interactive


In [1]:
# imports we will use
import altair as alt
import pandas as pd

In [2]:
alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

In [3]:
df_fold = pd.read_csv(
    "https://raw.githubusercontent.com/hsinhu/si649project/master/data/winterOlymic_more_country_medals_fold.csv")


In [4]:
winterOlymic_country_population = pd.read_csv("https://raw.githubusercontent.com/hsinhu/si649project/master/data/winterOlymic_country_related_to_population.csv")
winterOlymic_country_population.head()

,country,No. Winter Games,Gold,Silver,Bronze,Total,population,Weighted medals,Weighted medals per million people,Total medals per million people,Gold per million people,Silver per million people,Bronze per million people
0,Norway,23,132,125,111,368,5005700,889,177.597539,73.516192,26.369938,24.971532,22.174721
1,United States,23,105,110,90,305,313382000,730,2.329425,0.973253,0.335054,0.351009,0.287189
2,Germany,12,92,88,60,240,81831000,604,7.381066,2.932874,1.124268,1.075387,0.733218
3,Soviet Union,9,78,57,59,194,272000000,485,1.783088,0.713235,0.286765,0.209559,0.216912
4,Canada,23,73,64,62,199,34771400,482,13.861967,5.723094,2.099427,1.840593,1.783075


In [5]:
!pip install altair_saver

In [6]:
##################### Figure 1 (refined) #############################
medals = alt.Chart(df_fold).transform_joinaggregate(
    groupby=['country'],
    Total_count_medal='sum(value):Q'
).mark_bar().encode(
    alt.X("value:Q", stack=True, title='Number of Total Medals', axis=None),
    alt.Y("country:N",title='', sort=alt.EncodingSortField(
        field="Total_count_medal", order="descending"),axis = None),
    alt.Color('Medal_type:N',  scale=alt.Scale(domain=['Gold', 'Silver', 'Bronze'], range=['gold', 'silver', '#CD7F32'])),
    order=alt.Order('order:Q', sort='ascending')
).transform_window(
    sort=[alt.SortField('Total_count_medal',order='descending')],
    Total_avg_rank='rank(*)'
).transform_filter(
    alt.datum.Total_avg_rank <= 30
)

medals_annotation1 = alt.Chart(df_fold).transform_joinaggregate(
    groupby=['country'],
    Total_count_medal='sum(value):Q'
).mark_text(
    dx=-15, 
    color='white',
    size=15,
    align='center',
    baseline='middle',
).encode(
    alt.X("value:Q", stack=True,  axis=None),
    alt.Y("country:N",title='Country', sort=alt.EncodingSortField(
    field="Total_count_medal", order="descending"), axis = None),
    detail='Medal_type:N',
    text=alt.Text('value:Q'),
    order=alt.Order('order:Q', sort='ascending')

).transform_window(
    sort=[alt.SortField('Total_count_medal',order='descending')],
    Total_avg_rank='rank(*)'
).transform_filter(
    alt.datum.Total_avg_rank <= 30
)

y_lable1 = alt.Chart(df_fold).transform_joinaggregate(
    groupby=['country'],
    Total_count_medal='sum(value):Q'
).transform_window(
    sort=[alt.SortField('Total_count_medal',order='descending')],
    Total_avg_rank='rank(*)'
).transform_filter(
    alt.datum.Total_avg_rank <= 30
).mark_text(
    # dx=25, 
    color='black',
    size=15,
    width = 15,
    align='right',
    baseline='middle',
).encode(
    alt.Y("country:N",title='', sort=alt.EncodingSortField(
        field="Total_count_medal", order="descending"), axis = None),
    text=alt.Text('country:N'),
)

y_lable1 = y_lable1.properties(
    height = 400,
    # width = 20,
)

medals_total_plus_anotate = (medals+ medals_annotation1).properties(
    title = "1924 - 2018 Winter Olympics Number of Total Medals for top 10 countries",
    height = 400,
    # width = 300,
)

##static version
# medals_total_static = (y_lable1 | medals_total_plus_anotate)

# medals_total_static.configure_axis(
#     labelFontSize=17,
#     titleFontSize=17
# ).configure_title(
#     fontSize=19,
#     anchor='start',
#     align = 'left',
# ).configure_legend(
#     titleFontSize=17,
#     labelFontSize=17,
#     orient='none',
#     legendX	= 700,
#     legendY = 20,
# ).configure_scale(
#     bandPaddingInner = 0.25
# ).configure_view(
#     strokeOpacity=0
# )

In [7]:
selection2 = alt.selection_multi(empty="all", on="mouseover")
selection1 = alt.selection_multi(fields=['Medal_type'], bind='legend')

opacityCondition2 = alt.condition(selection2, alt.value(1), alt.value(0.3))

medals_inter = medals.add_selection(
    selection1,
    selection2

).encode(
    opacity = opacityCondition2
).transform_filter(
    selection1,
)

medals_annotation1_inter = medals_annotation1.add_selection(
    # selection2
).encode(
    # opacity = opacityCondition2,
).transform_filter(
    selection1
)

y_lable1_inter = y_lable1.add_selection(
     selection2
).encode(
    opacity = opacityCondition2,
    # color =colorCondition2 
).transform_filter(
    selection1
)


medals_total_plus_anotate_inter = (medals_inter + medals_annotation1_inter).encode(
    tooltip = ["country:N","Medal_type:N", "value:Q", "Total_count_medal:Q"],
).properties(
    title = "1924 - 2018 Winter OlympicsTotal Medals top 10 countries",
    height = 400,
    # width = 600,
)

medals_all_inter = (y_lable1_inter | medals_total_plus_anotate_inter)

vis1 = medals_all_inter

vis1 = vis1.configure_scale(
    bandPaddingInner = 0.25
).configure_title(
    anchor='start',
    fontSize=15,
).configure_view(
    strokeOpacity=0
).configure_legend(
    orient='bottom',
)

vis1
# vis1.save("vis1.json")


alt.HConcatChart(...)

In [8]:
###### figure 2 static############
medals_weighted = alt.Chart(winterOlymic_country_population).mark_bar(strokeWidth=10).encode(
    alt.X("Weighted medals:Q", title='Number of Weighted Medals', axis = None),
    alt.Y("country:N",title='', sort=alt.EncodingSortField(
        field="Weighted medals", order="descending"),axis = None),
        color = "Weighted medals:Q"
).transform_window(
    sort=[alt.SortField('Weighted medals',order='descending')],
    Total_avg_rank='rank(*)'
)


y_lable = alt.Chart(winterOlymic_country_population).transform_window(
    sort=[alt.SortField('Weighted medals',order='descending')],
    Total_avg_rank='rank(*)'
).mark_text(
    # dx=25, 
    color='black',
    size=14,
).encode(
    alt.Y("country:N",title='', sort=alt.EncodingSortField(
        field="Weighted medals", order="descending"), axis = None),
    text=alt.Text('country:N'),
)

medals_weighted_annotation = alt.Chart(winterOlymic_country_population).mark_text(
    dx=25, 
    color='black',
    size=15,
    align='center',
    baseline='middle',
).encode(
    alt.X("Weighted medals:Q", title='Number of Weighted Medals'),
    alt.Y("country:N",title='', sort=alt.EncodingSortField(
        field="Weighted medals", order="descending"), axis = None),
    text=alt.Text('Weighted medals:Q'),
).transform_window(
    sort=[alt.SortField('Weighted medals',order='descending')],
    Total_avg_rank='rank(*)'
)


medals_plus_anotate = (medals_weighted + medals_weighted_annotation).properties(
    title = "1924 - 2018 Winter Olympics Number of Weighted Medals for top 10 countries",
    height = 400,
    # width = 180,
)

y_lable = y_lable.properties(
    height = 400,
)

# static combinataion
# medals_static = (y_lable | medals_plus_anotate).resolve_scale(y = 'shared')
# medals_static.configure_axis(
#     labelFontSize=17,
#     titleFontSize=17
# ).configure_title(
#     fontSize=24,
#     anchor='start',
#     align = 'left',
# ).configure_legend(
#     titleFontSize=17,
#     labelFontSize=17,
#     orient='none',
#     legendX	= 700,
#     legendY = 20,
# ).configure_scale(
#     bandPaddingInner = 0.25
# ).configure_view(
#     strokeOpacity=0
# )

In [9]:
###### figure 2 inter############

rank_options = [10, 15, 20]
rank_radio = alt.binding_radio(options = rank_options)
radio_selector = alt.selection_single(
    name = "rank",
    fields=["rank"],
    bind=rank_radio,
    init = {"rank": 10}
)


selection2 = alt.selection_multi(empty="all", on="mouseover")

colorCondition = alt.condition(selection2, "Weighted medals:Q", alt.value("grey"))
opacityCondition = alt.condition(selection2, alt.value(1), alt.value(0))
opacityCondition1 = alt.condition(selection2, alt.value(1), alt.value(0.8))
opacityCondition2 = alt.condition(selection2, alt.value(1), alt.value(0.5))

medals_weighted_inter = medals_weighted.add_selection(
    # selection2
).encode(
    color = colorCondition,
    opacity = opacityCondition1
).transform_filter(
    alt.datum.Total_avg_rank <= radio_selector.rank
)

medals_weighted_annotation_inter = medals_weighted_annotation.add_selection(
    # selection2
).encode(
    opacity = opacityCondition2,
).transform_filter(
    alt.datum.Total_avg_rank <= radio_selector.rank
)


y_lable_inter = y_lable.add_selection(
     selection2,
     radio_selector
).encode(
    opacity = opacityCondition2).transform_filter(
    alt.datum.Total_avg_rank <= radio_selector.rank
)

medals_plus_anotate_inter = (medals_weighted_inter + medals_weighted_annotation_inter).encode(
    tooltip = ["country:N", "Weighted medals:Q"]
    
).properties(
    title = "1924 - 2018 Winter Olympics Weighted Medals top 10 countries",
    # width = 400
).add_selection(
    selection2,
    radio_selector
)

medals_inter = (y_lable_inter | medals_plus_anotate_inter).resolve_scale(y = 'shared')

vis2 = medals_inter.configure_scale(
    bandPaddingInner = 0.25
).configure_title(
    anchor='start',
    fontSize=15,
).configure_view(
    strokeOpacity=0
).configure_legend(
    orient='bottom',
)
vis2 
# vis2.save("vis2.json")

alt.HConcatChart(...)

In [10]:
vis1.save("vis1.json")
vis2.save("vis2.json")